### 8.7 원근 투시(투영) 변환

In [2]:
# 8.7.1 원근 왜곡 보정
import numpy as np, cv2

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

pts1 = np.float32([(80, 40), (315, 133), (75, 300), (335, 300)]) # 원본 점 4개
pts2 = np.float32([(50, 60), (340, 60), (50, 320), (340, 320)]) # 결과 점 4개   

perspect_mat = cv2.getPerspectiveTransform(pts1, pts2)
dst = cv2.warpPerspective(image, perspect_mat, image.shape[1::-1], cv2.INTER_CUBIC)
print('perspect_mat = \n%s\n' % perspect_mat)

ones = np.ones((4, 1), np.float64)
pts3 = np.append(pts1, ones, axis=1)
pts4 = cv2.gemm(pts3, perspect_mat.T, 1, None, 1)

print("원본 영상 좌표 \t 목적 영상 좌표 \t 동차 좌표 \t 변환 결과 좌표")
for i in range(len(pts4)):
    pts4[i] /= pts4[i][2]
    print("%i : %-14s %-14s %-18s %-18s" % (i, pts1[i], pts2[i], pts3[i], pts4[i]))
    cv2.circle(image, tuple(pts2[i].astype(int)), 3, (0, 255, 0), -1)
    cv2.circle(dst, tuple(pts2[i].astype(int)), 3, (0, 255, 0), -1)

cv2.imshow('image', image)
cv2.imshow('dst', dst)
cv2.waitKey(0)

perspect_mat = 
[[ 6.25789284e-01  3.98298577e-02 -6.88839366e+00]
 [-5.02676539e-01  1.06358288e+00  5.13923399e+01]
 [-1.57086418e-03  5.25700042e-04  1.00000000e+00]]

원본 영상 좌표 	 목적 영상 좌표 	 동차 좌표 	 변환 결과 좌표
0 : [80. 40.]      [50. 60.]      [80. 40.  1.]      [50. 60.  1.]     
1 : [315. 133.]    [340.  60.]    [315. 133.   1.]   [340.  60.   1.]  
2 : [ 75. 300.]    [ 50. 320.]    [ 75. 300.   1.]   [ 50. 320.   1.]  
3 : [335. 300.]    [340. 320.]    [335. 300.   1.]   [340. 320.   1.]  


-1

In [1]:
import numpy as np, cv2
from Common.functions import contain_pts

def draw_rect(img):
    rois = [(p - small, small * 2) for p in pts1]
    for (x, y), (w, h) in np.int32(rois):
        roi = img[y:y+h, x:x+w]
        val = np.full(roi.shape, 80, np.uint8)
        cv2.add(roi, val, roi)
        cv2.rectangle(img, (x, y, w, h), (0, 255, 0), 1)
    cv2.polylines(img, [pts1.astype(int)], True, (0, 255, 0), 1)
    cv2.imshow('select rect', img)

def warp(img):
    perspect_mat = cv2.getPerspectiveTransform(pts1, pts2)
    dst = cv2.warpPerspective(img, perspect_mat, (350, 400), cv2.INTER_CUBIC)
    cv2.imshow("Perspective Transform", dst)

def onMouse(event, x, y, flags, param):
    global check
    if event == cv2.EVENT_LBUTTONDOWN:
        for i, p in enumerate(pts1):
            p1, p2 = p - small, p + small
            if contain_pts((x, y), p1, p2): check = i

    if event == cv2.EVENT_LBUTTONUP: check = -1

    if check >= 0:
        pts1[check] = (x, y)
        draw_rect(np.copy(image))
        warp(np.copy(image))

image = cv2.imread("images_08/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

small = np.array([12, 12])
check = -1
pts1 = np.float32([(100, 100), (300, 100), (300, 300), (100, 300)])
pts2 = np.float32([(0, 0), (400, 0), (400, 350), (0, 350)])

draw_rect(np.copy(image))
cv2.setMouseCallback('select rect', onMouse, 0)
cv2.waitKey(0)

-1